In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

## Glipse Data

Read data

In [ ]:
btc = pd.read_csv('/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv')
# Converting the Timestamp column from string to datetime
btc['Timestamp'] = [datetime.fromtimestamp(x) for x in btc['Timestamp']]
btc = btc.set_index('Timestamp')

#### Time resampling

In [ ]:
btc_daily = btc.resample("24H").mean() #daily resampling

#### Time Series Decomposition & Statistical Tests

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
def fill_missing(df):
    ### function to impute missing values using interpolation ###
    df['Open'] = df['Open'].interpolate()
    df['Close'] = df['Close'].interpolate()
    df['Weighted_Price'] = df['Weighted_Price'].interpolate()
    df['Volume_(BTC)'] = df['Volume_(BTC)'].interpolate()
    df['Volume_(Currency)'] = df['Volume_(Currency)'].interpolate()
    df['High'] = df['High'].interpolate()
    df['Low'] = df['Low'].interpolate()
    print(df.head())
    print(df.isnull().sum())
    
fill_missing(btc_daily)

In [ ]:

plt.figure(figsize=(15,12))
series = btc_daily.Weighted_Price
result = seasonal_decompose(series, model='additive',period=1)
result.plot()

In [ ]:
!pip install neuralprophet[live]
from neuralprophet import NeuralProphet

In [ ]:
# Renaming the column names accroding to Prophet's requirements

daily_data_fb = btc_daily.reset_index()[['Timestamp','Weighted_Price']].rename({'Timestamp':'ds','Weighted_Price':'y'}, axis=1)
daily_data_fb.head()

In [ ]:
test_length = 365
split_date = "2020-01-01"
train_filt = daily_data_fb['ds'] <= split_date
test_filt = daily_data_fb['ds'] > split_date

train_fb = daily_data_fb[train_filt]
test_fb = daily_data_fb[test_filt]

print("train data shape :", train_fb.shape[0] / daily_data_fb.shape[0] * 100)
print("test data shape :", test_fb.shape[0] / daily_data_fb.shape[0] * 100)

In [ ]:
nprophet_model = NeuralProphet()
metrics = nprophet_model.fit(train_fb, freq="D")
future_df = nprophet_model.make_future_dataframe(train_fb, 
                                                 periods = test_length, 
                                                 n_historic_predictions=len(train_fb))
preds_df_2 = nprophet_model.predict(future_df)

In [ ]:
nprophet_model.plot(preds_df_2);

In [ ]:
nprophet_model.plot_components(preds_df_2, residuals=True);

In [ ]:
nprophet_model.plot_parameters();